In [1]:
import torch
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from tqdm import tqdm
import shutil
import pandas as pd
import numpy as np
import os
torch.manual_seed(1234)
from evaluate_metrics import compute_eer
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 14}

In [2]:
# load eval set
eval_df = pd.read_csv("/datab/Dataset/ASVspoof/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval.trl.txt", sep=" ", header=None)
eval_df.columns = ["sid", "utt","phy", "attack", "label"]
eval_df.head


<bound method NDFrame.head of            sid           utt phy attack     label
0      LA_0039  LA_E_2834763   -    A11     spoof
1      LA_0014  LA_E_8877452   -    A14     spoof
2      LA_0040  LA_E_6828287   -    A16     spoof
3      LA_0022  LA_E_6977360   -    A09     spoof
4      LA_0031  LA_E_5932896   -    A13     spoof
...        ...           ...  ..    ...       ...
71232  LA_0004  LA_E_1665632   -      -  bonafide
71233  LA_0038  LA_E_5085671   -    A09     spoof
71234  LA_0012  LA_E_4926022   -    A16     spoof
71235  LA_0052  LA_E_2894498   -      -  bonafide
71236  LA_0009  LA_E_4689563   -    A19     spoof

[71237 rows x 5 columns]>

In [7]:
eval_df['attack'].unique()

array(['A11', 'A14', 'A16', 'A09', 'A13', '-', 'A12', 'A18', 'A15', 'A08',
       'A17', 'A10', 'A07', 'A19'], dtype=object)

In [10]:
pred_df = pd.read_csv("asvspoof2021/LA/wav2vec-bio/asvspoof2019_21.txt", sep=" ", header=None)
pred_df.columns = ["utt", "asdf", "sefsef", "score"]


# merge eval_df and pred_df on utt
res_df = pd.merge(eval_df, pred_df, on='utt')
# print(res_df.head)

# calcuate EER by attack type
eer_dict = {}
# res_df['attack'].unique()
for attack in res_df['attack'].unique():
    # print(attack)
    spoof_scores = res_df[(res_df['attack'] == attack) & (res_df['label'] == 'spoof')]['score']
    bonafide_scores = res_df[(res_df['label'] == 'bonafide')]['score']
    eer, threshold = compute_eer(bonafide_scores, spoof_scores)
    eer_dict[attack] = eer
    print("{}: EER: {:.4f}%, threshold: {:.4f}".format(attack, eer*100, threshold))

# compute EER
# spoof_scores = res_df[res_df['label'] == 'spoof']['score']
# bonafide_scores = res_df[res_df['label'] == 'bonafide']['score']
# eer, threshold = compute_eer(bonafide_scores, spoof_scores)
# print("EER: {:.4f}%, threshold: {:.4f}".format(eer*100, threshold))
# res_df['pred'] = res_df['score'].apply(lambda x: 'spoof' if x < threshold else 'bonafide')
# # confusion matrix
# cm = confusion_matrix(res_df["label"], res_df["pred"], labels=["spoof","bonafide"])
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["spoof","bonafide"])
# disp.plot(cmap='Greens', values_format='g')
# plt.title("wav2vec-bio_asvspoof2019_21.txt")
# # plt.savefig("figs/ori_assl_cm.png", dpi=300)
# plt.show()


A11: EER: 0.1459%, threshold: -0.0012
A14: EER: 0.0645%, threshold: -5.1865
A16: EER: 0.0645%, threshold: -4.8951
A09: EER: 0.0577%, threshold: -6.8903
A13: EER: 0.0577%, threshold: -6.0583
-: EER: nan%, threshold: -10.2035
A12: EER: 0.0645%, threshold: -3.3135
A18: EER: 5.4736%, threshold: -0.0000
A15: EER: 0.0645%, threshold: -4.6514
A08: EER: 0.0577%, threshold: -7.3271
A17: EER: 1.5065%, threshold: -0.0001
A10: EER: 0.2613%, threshold: -0.0003
A07: EER: 0.0577%, threshold: -6.4221
A19: EER: 9.0122%, threshold: -0.0000


In [ ]:
# break down based on attack type
